

---



# План анализа данных

  1. Загрузить данные для обучения;
  2. Обработать данные перед обучением модели;
  3. Обучить и провалидировать модель;
  4. Попробовать улучшить модель с помощью подбора параметров.



---

# Установка библиотек

In [1]:
# Grab Jaime's excellent condacolab package: https://github.com/jaimergp/condacolab
# Note: you should probably read the README file at that repo.
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [2]:
!mamba install -c conda-forge rdkit chembl_structure_pipeline


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit', 'chembl_structure_pipeline']

conda-forge/linux-64     Using cache
conda-forge/noarch     

In [3]:
!pip install git+https://github.com/bp-kelley/descriptastorus

  Cloning https://github.com/bp-kelley/descriptastorus to /tmp/pip-req-build-4baj1p8x
  Running command git clone -q https://github.com/bp-kelley/descriptastorus /tmp/pip-req-build-4baj1p8x


# Импорт библиотек

In [4]:
import pandas as pd
import os

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split

from typing import Dict, List, Optional
from pathlib import Path

from multiprocessing.pool import Pool

from rdkit.Chem.MolStandardize.tautomer import TautomerCanonicalizer



# Загрузка данных

In [5]:
# 2й датасет - esol
# 1й -  freesolv (раскомментируйте соответственно)
#!wget https://www.dropbox.com/s/01wcrxi8qb6he1b/SAMPL.csv
!wget https://raw.githubusercontent.com/kostyastrong/HSE_nlp_mol_properties/master/data/esol.csv

--2021-05-09 21:35:33--  https://raw.githubusercontent.com/kostyastrong/HSE_nlp_mol_properties/master/data/esol.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96699 (94K) [text/plain]
Saving to: ‘esol.csv’

esol.csv            100%[===================>]  94.43K  --.-KB/s    in 0.02s   

2021-05-09 21:35:33 (5.41 MB/s) - ‘esol.csv’ saved [96699/96699]



In [6]:
class DatasetsHolder:
    @staticmethod
    def read_datasets(inp_folder_path):
        # return pandas DataFrame
        return pd.read_csv(inp_folder_path)


class StandardizeDatasets:
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
        "crete typical standartization of one smiles"
        mol = Chem.MolFromSmiles(smi)
        return Chem.MolToSmiles(mol)

    def standardize(self, inp_path: Path, out_path: Path):
        "apply standartization to all smiles"
        dataset = DatasetsHolder()
        df = dataset.read_datasets(inp_path)
        # df['standardize_smiles'] = df['smiles'].apply(self.standardize_smiles)
        with Pool(10) as pool:
          df['standardize_smiles'] = list(
                      tqdm(pool.imap(self.standardize_smiles, df.smiles), total=df.shape[0])
                  )
        df.to_csv(out_path)


class StandardizeTautomers(StandardizeDatasets):
    @staticmethod
    def standardize_smiles(smi: str) -> Optional[str]:
      "apply TautomerCanonicalizer() to standartization"
      tc = TautomerCanonicalizer()
      mol = Chem.MolFromSmiles(smi)
      return Chem.MolToSmiles(tc.canonicalize(mol))

In [7]:
st = StandardizeTautomers()
st.standardize('esol.csv', 'out_esol.csv')

100%|██████████| 1128/1128 [00:14<00:00, 78.65it/s]


In [8]:
SMILES_COLUMN = 'standardize_smiles'
VALUE_COLUMN = 'expt'

DATASET_INPUT_PATH = 'out_esol.csv'

# Выделение признаков

In [9]:
data_reader = DatasetsHolder
data = data_reader.read_datasets(DATASET_INPUT_PATH)

In [10]:
from descriptastorus.descriptors import rdDescriptors
from rdkit import Chem
import logging


generator = rdDescriptors.RDKit2D()

In [11]:

def rdkit_2d_features(smiles: str):
    data = generator.process(smiles)
    if data[0]:
        return data[1:]
    else:
        print(f'{smiles} not processed')
        return None
    

In [12]:
def create_feature_dataframe(df):
    features_names = [i[0] for i in generator.columns]
    features = []
    for j in range(len(df)):
        features.append(dict(zip(features_names, (x for x in rdkit_2d_features(df[SMILES_COLUMN].iloc[j])))))
    data_proc = pd.DataFrame(features)
    data_proc['target'] = df['measured log solubility in mols per litre']
    return data_proc
    


In [13]:
create_feature_dataframe(data)


,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,ExactMolWt,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,FractionCSP3,HallKierAlpha,HeavyAtomCount,HeavyAtomMolWt,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,MaxAbsEStateIndex,MaxAbsPartialCharge,...,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed,target
0,1.654937,759.662938,23.413485,16.862520,16.862520,15.277295,9.998816,9.998816,7.601218,7.601218,5.431494,5.431494,3.506930,3.506930,80.729515,41.007583,0.0,0.000000,5.563451,0.000000,0.000000,30.331835,6.069221,0.000000,18.947452,457.158411,0.812500,1.375000,1.968750,0.650000,-1.73,32,430.216,1.212120e+07,24.903474,10.926356,5.251706,182.935327,10.253329,0.393567,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.217518,-0.770
1,2.148162,459.484175,10.673362,8.357948,8.357948,7.270857,4.676643,4.676643,3.210611,3.210611,2.135103,2.135103,1.340444,1.340444,0.000000,4.794537,0.0,5.907180,11.323699,5.687386,6.263163,12.990104,30.331835,5.316789,4.417151,201.078979,1.200000,1.933333,2.533333,0.083333,-2.03,15,190.137,4.231896e+03,9.522160,4.002882,2.070849,87.724095,11.724911,0.468799,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.811283,-3.300
2,3.625760,171.311799,8.690234,7.554513,7.554513,5.163902,3.908188,3.908188,2.969252,2.969252,1.443820,1.443820,0.788002,0.788002,0.000000,4.794537,0.0,0.000000,0.000000,24.700908,5.573105,6.076020,6.923737,19.923495,0.000000,152.120115,1.272727,1.909091,2.363636,0.500000,-0.85,11,136.109,2.036951e+02,10.150000,5.899351,7.042356,68.806046,10.020498,0.298566,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.343706,-2.060
3,2.041379,1071.547817,14.518297,12.082904,12.082904,10.915816,7.636751,7.636751,5.829201,5.829201,4.648219,4.648219,3.586716,3.586716,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,43.089794,0.000000,0.000000,84.929139,0.000000,278.109550,0.272727,0.636364,1.136364,0.000000,-2.86,22,264.242,2.961396e+05,11.762233,4.315741,1.523286,128.158061,2.270278,0.061629,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.291526,-7.870
4,3.125000,60.124818,3.535534,2.717649,3.534146,2.500000,1.471405,2.414214,0.793148,1.609645,0.425381,1.053920,0.226805,0.680414,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,11.336786,22.892860,0.000000,0.000000,84.003371,1.000000,1.600000,1.800000,0.000000,-0.30,5,80.111,2.288644e+01,2.912766,1.221050,0.484065,35.071766,2.041667,0.152454,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.448927,-1.330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,3.541197,58.445472,6.077350,2.967173,5.309098,2.943376,1.292058,2.644169,0.842309,2.721600,0.247436,1.014011,0.000000,0.000000,10.462654,13.171245,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.530884,0.000000,195.890224,1.428571,1.571429,1.571429,1.000000,0.56,7,196.373,1.708617e+01,7.560000,2.218900,3.169756,51.771613,10.999421,0.414120,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.523506,-1.710
1124

In [14]:
data.head()

,Unnamed: 0,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles,standardize_smiles
0,0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c...
1,1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2,Cc1occc1C(=O)Nc1ccccc1
2,2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O),CC(C)=CCCC(C)=CC=O
3,3,Picene,-6.618,2,278.354,0,5,0,0.00,-7.87,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21
4,4,Thiophene,-2.232,2,84.143,0,1,0,0.00,-1.33,c1ccsc1,c1ccsc1


In [15]:
data_feats = create_feature_dataframe(data)

# Деление данных на тренировочну и тестовую выборки

In [16]:
train_data, test_data = train_test_split(data_feats, test_size=0.3)

In [17]:
X_train = train_data.drop(columns='target')
y_train = train_data.target
X_test = test_data.drop(columns='target')
y_test = test_data.target


# Создание модели

In [18]:
from xgboost import XGBRegressor

In [19]:
boosting_tree_model = XGBRegressor()

In [20]:
boosting_tree_model.fit(X_train, y_train)
print(boosting_tree_model.score(X_test, y_test))

[21:36:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9033894590348397


# Обучение и валидация

![alt text](https://drive.google.com/uc?id=1Ilkmp248M0kKA3wFJQNQcNEY9OFsVoWz)

Стандартно для правильной валидации модели используют отложенную выборку. То есть мы разбиваем наши данные на **тренировочную** выборку, **тестовую** выборку и **отложенную** выборку. Соответственно, обучаем модель на тренировочной, в ходе обучения проверяем результат на тестовой выборке, а в конце обучения, чтобы оценить качество модели, ошибку считаем на отложенной выборке.

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1wgVvskPBQJgiRwpsHUmUOS-MhfBatsWy" 
alt="IMAGE ALT TEXT HERE" width="480" border="0" /></a>

*Замечание:* тестовая и отложенная выборка могут совпадать. Главное - на этой части данных модель не обучается!


Однако, при таком подходе в обучении модели участвует только тренировочная выборка. Тестовую и отложенную мы используем только для проверки. Если у нас мало данных - это непозволительная роскошь. 

Другой популярный подход это **кросс-валидация** или скользящий контроль. Суть метода заключается в том, что мы делаем не одно разбиение датесета, а несколько разбиений таким образом, чтобы все данные использовались и в обучении и для проверки. Такие разбиения называются **фолдами**. 

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=14fZpuBDsTMqv1XtLJvcKMNNa1vlr_ZG6" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>


Преимущества такого подхода:
* используем все данные для обучения;
* можем оценить устойчивость модели. Если ошибки полученные на разных фолдах сильно отличаются, что модель неустойчива.

Недостаток метода в том, что нам нужно обучать не одну модель, а несколько (столько, сколько мы выбрали фолдов).

На практике часто выбирают 5 фолдов.

In [21]:
from sklearn.model_selection import KFold, cross_val_score

In [22]:
kf = KFold(n_splits = 5)

In [23]:
boosting_tree_model = XGBRegressor()

In [24]:
rmse_scores = cross_val_score(boosting_tree_model, X_train, y_train, cv=kf)

[21:36:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:36:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Функция `cross_val_score` воспроизводит разбиение, обучение и тестирование в соответствие с типом и параметрами передаваемого в нее валидатора. 

В нее передаем оцениваемую модель, таблицу входных данных, выходную переменную, способ разделения данных (фолды) и метрику, которую мы хотим оценить. В данном случае мы хотим оценить **r2_score**.

На выходе получим значения метрик. Так как мы передали в `KFold` с параметром **n_splits=5**, то и значений мы получим **5**.

In [25]:
rmse_scores.mean()

0.898884485445729

# Поиск по сетке



<a href="https://drive.google.com/uc?id=1Goc0VR5I--q9rYj-vYlmddanKP3-3sLJ
" target="_blank"><img src="https://drive.google.com/uc?id=1Goc0VR5I--q9rYj-vYlmddanKP3-3sLJ" 
alt="IMAGE ALT TEXT HERE" width="480" border="0" /></a>


Теперь, когда у нас есть надёжный способ оценивать качество модели, мы можем перейти к подбору гиперпараметров модели.

Чтобы выработать некоторую интуицию о самых важных параметрах градиентного бустинга на решающих деревьях, сначала мы попробуем в ручную поменять их и посмотреть, как меняются метрики. 



In [33]:
def hyperparamters_search(max_depth, n_estimators, points, target):
  # инициализируем модель и способ валидации k-fold

  # считаем метрики в процессе кросс-валидации
    mae_scores = 1
    rmse_scores = 2
    r2_scores = 1

  # считаем среднее по полученным результатам
    mae = 1
    rmse = 2
    r2 = 1

    print("MAE: {0:7.2f}, RMSE: {1:7.2f}, R2: {2:3.2f} for xgboost model".format(mae, rmse, r2))

## Поиск по сетке. 

Вместо того, чтобы перебирать параметры руками, можно использовать метод **поиска по сетке** (Grid Search). В процессе поиска по сетке мы указываем варианты каждого из параметров, которые хотим перебрать, а функция смотрит на все их возможные варианты и выдает лучший набор в зависимости от выбранной метрики. Например, на картинке ниже перебираются параметры "регуляризация" и "скорость обучения".

<a href="https://drive.google.com/uc?id=1jAZLpihYxu_FPvN9PIJ1G4S_KvO_6Ku6
" target="_blank"><img src="https://drive.google.com/uc?id=1FhZpRMWuzCXQs1DDdTn11hjmH3MS6C6j" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

Поиск по сетке реализован в **sklearn**, импортируем его:

In [29]:
from sklearn.model_selection import GridSearchCV


In [31]:
boosting_tree_model = XGBRegressor()
param = {"n_estimators": range(380, 420, 10), 'max_depth': range(40, 66, 5)}
clf = GridSearchCV(boosting_tree_model, param)
clf.fit(X_train, y_train)

[21:37:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:37:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:37:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:37:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:38:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:38:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:38:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:38:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': range(40, 66, 5),
                         'n_estimators': range(380, 420, 10)},
          

In [32]:
clf.best_score_

0.8877781406860741